# Colab setup

In [1]:
# if 'google.colab' in str(get_ipython()):
#     !pip install pytorch-lifestream

In [2]:
# !pip install lightning

## Data load

In [8]:
# import os

# if not os.path.exists('data/rosbank/train.csv'):
#     !mkdir -p data/rosbank
#     !curl -OL https://storage.yandexcloud.net/di-datasets/rosbank-ml-contest-boosters.pro.zip
#     !unzip -j -o rosbank-ml-contest-boosters.pro.zip 'data/*.csv' -d data/rosbank
#     !mv age-prediction-nti-sbebank-2019.zip data/rosbank/

The syntax of the command is incorrect.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 29 8979k   29 2683k    0     0  2683k      0  0:00:03 --:--:--  0:00:03 2958k
 54 8979k   54 4931k    0     0  4931k      0  0:00:01  0:00:01 --:--:-- 2586k
 71 8979k   71 6409k    0     0  3204k      0  0:00:02  0:00:02 --:--:-- 2204k
 85 8979k   85 7670k    0     0  2556k      0  0:00:03  0:00:03 --:--:-- 1963k
 98 8979k   98 8825k    0     0  2206k      0  0:00:04  0:00:04 --:--:-- 1798k
100 8979k  100 8979k    0     0  1795k      0  0:00:05  0:00:05 --:--:-- 1532k
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.


## Setup

In [99]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import pytorch_lightning as pl

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split

from models import GptPretrainContrastiveModule, NextItemPredictionModule

from ptls.nn import TrxEncoder, RnnEncoder
from ptls.frames.gpt import GptPretrainModule, GptDataset
from ptls.preprocessing import PandasDataPreprocessor
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames import PtlsDataModule

from dataset import MlmNoSliceDataset

import os
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data preproccessing

In [100]:
data_path = 'data/rosbank'

source_data = pd.read_csv(os.path.join(data_path, 'train.csv'))
source_data['TRDATETIME'] = pd.to_datetime(source_data['TRDATETIME'], format='%d%b%y:%H:%M:%S')
source_data = source_data.sort_values(by=['TRDATETIME'])
source_data = source_data.rename(columns={'cl_id':'client_id', 'MCC':'small_group', 'amount':'amount_rur'})
source_data.head()

,PERIOD,client_id,small_group,channel_type,currency,TRDATETIME,amount_rur,trx_category,target_flag,target_sum
26790,01/10/2016,485,4121,type2,810,2016-10-07 00:00:00,242.0,POS,1,85.00
72077,01/10/2016,1290,5411,type2,810,2016-10-07 00:00:00,2465.0,POS,1,321242.09
26988,01/10/2016,485,6011,type2,810,2016-10-07 00:00:00,3600.0,WD_ATM_PARTNER,1,85.00
72068,01/10/2016,1290,6011,type2,810,2016-10-07 18:57:17,10000.0,WD_ATM_ROS,1,321242.09
189585,01/10/2016,3351,8999,type3,810,2016-10-08 00:00:00,10000.0,POS,0,0.00


In [101]:
mcc_to_id = {mcc: i+1 for i, mcc in enumerate(source_data['small_group'].unique())}

source_data['amount_rur_bin'] = 1 + KBinsDiscretizer(10, encode='ordinal', subsample=None).fit_transform(source_data[['amount_rur']]).astype('int')
source_data['small_group'] = source_data['small_group'].map(mcc_to_id)

In [102]:
preprocessor = PandasDataPreprocessor(
    col_id='client_id',
    col_event_time='TRDATETIME',
    event_time_transformation='dt_to_timestamp',
    cols_category=['small_group'],
    cols_numerical=['amount_rur_bin'],
    return_records=True,
)

In [103]:
%%time

dataset = preprocessor.fit_transform(source_data[['client_id', 'TRDATETIME', 'small_group', 'amount_rur_bin']])

CPU times: total: 2.14 s
Wall time: 2.18 s


In [104]:
dataset = sorted(dataset, key=lambda x: x['client_id'])

In [105]:
train_valid, test = train_test_split(dataset, test_size=0.2, random_state=42)

train, valid = train_test_split(train_valid, test_size=0.2, random_state=42)
len(train), len(valid), len(test)

(3200, 800, 1000)

In [8]:
train_dl = PtlsDataModule(
    train_data=GptDataset(
        MemoryMapDataset(
            data=train,
            # i_filters=[
            #     SeqLenFilter(min_seq_len=25),
            # ],
        ),
        min_len=25, 
        max_len=200
    ),
    valid_data=MlmNoSliceDataset(
        MemoryMapDataset(
            data=valid,
            # i_filters=[
            #     SeqLenFilter(min_seq_len=25),
            # ],
        ),
    ),
    test_data=MlmNoSliceDataset(
        MemoryMapDataset(
            data=test,
            # i_filters=[
            #     SeqLenFilter(min_seq_len=25),
            # ],
        ),
    ),
    train_batch_size=128,
)

## Embedding training (representation)

Model training in our framework organised via pytorch-lightning (pl) framework.
The key parts of neural networks training in pl are: 

    * model (`pytorch_lightning.LightningModule`)
    * data loader (`torch.utils.data.DataLoader`)
    * trainer (`pytorch_lightning.Trainer`)
    
For futher details check https://pytorchlightning.ai/

In [19]:
print("Number of unique MCC codes:", source_data['small_group'].max())

Number of unique MCC codes: 344


### Model definition

In [20]:
trx_encoder_params = dict(
    embeddings_noise=0.0,
    embeddings={
        'small_group': {'in': 350, 'out': 16},
        'amount_rur_bin':{'in': 60, 'out': 16}
    },
)

seq_encoder = RnnEncoder(
        input_size=32,
        hidden_size=32,
        type='gru',
)

model = GptPretrainModule(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    seq_encoder=seq_encoder,
    max_lr=0.1
)

### Pre-training

In [21]:
trainer = pl.Trainer(
    max_epochs=100,
    gpus=1 if torch.cuda.is_available() else 0,
    callbacks=[pl.callbacks.EarlyStopping('gpt/valid_gpt_loss')],
    enable_progress_bar=True,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [22]:
%%time
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, train_dl)
print(trainer.logged_metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | trx_encoder    | TrxEncoder       | 6.6 K 
1 | _seq_encoder   | RnnEncoder       | 6.4 K 
2 | head           | ModuleDict       | 30.9 K
3 | loss           | CrossEntropyLoss | 0     
4 | train_gpt_loss | MeanMetric       | 0     
5 | valid_gpt_loss | MeanMetric       | 0     
----------------------------------------------------
43.8 K    Trainable params
0         Non-trainable params
43.8 K    Total params
0.175     Total estimated model params size (MB)


logger.version = 94


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\trainer.py:1933: PossibleUserWarning: The number of training 

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

{'gpt/loss': tensor(6.5488), 'gpt/valid_gpt_loss': tensor(6.5773), 'gpt/train_gpt_loss': tensor(6.5168)}
CPU times: total: 34.1 s
Wall time: 34 s


### Fine-tuning

In [23]:
model_downstream = NextItemPredictionModule(
    trx_encoder=TrxEncoder(**trx_encoder_params), # model.trx_encoder,
    seq_encoder=seq_encoder, # model._seq_encoder,
    target_col='small_group',
    max_lr=0.1
)

In [25]:
trainer = pl.Trainer(
    max_epochs=100,
    gpus=1 if torch.cuda.is_available() else 0,
    callbacks=[pl.callbacks.EarlyStopping('gpt/valid_f1_weighted', patience=5, mode='max')],
    enable_progress_bar=True,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [26]:
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model_downstream, train_dl)
print(trainer.logged_metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | trx_encoder    | TrxEncoder       | 6.6 K 
1 | _seq_encoder   | RnnEncoder       | 6.4 K 
2 | head           | Head             | 24.9 K
3 | loss           | CrossEntropyLoss | 0     
4 | train_gpt_loss | MeanMetric       | 0     
5 | valid_gpt_loss | MeanMetric       | 0     
----------------------------------------------------
37.8 K    Trainable params
0         Non-trainable params
37.8 K    Total params
0.151     Total estimated model params size (MB)


logger.version = 95


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\trainer.py:1933: PossibleUserWarning: The number of training batches (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

{'gpt/loss': tensor(2.8688), 'gpt/valid_gpt_loss': tensor(2.8537), 'gpt/valid_f1_weighted': tensor(0.2245, dtype=torch.float64), 'gpt/train_gpt_loss': tensor(2.8514)}


In [27]:
trainer.test(model_downstream, train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
  gpt/test_f1_weighted      0.2238036663493782
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'gpt/test_f1_weighted': 0.2238036663493782}]

## Embedding training (contrastive)

### Model definition

In [123]:
trx_encoder_params = dict(
    embeddings_noise=0.0,
    embeddings={
        'small_group': {'in': 350, 'out': 16},
        'amount_rur_bin':{'in': 11, 'out': 16}
    },
)

seq_encoder = RnnEncoder(
        input_size=32,
        hidden_size=32,
        type='gru',
)

model = GptPretrainContrastiveModule(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    seq_encoder=seq_encoder,
    max_lr=0.01,
    total_steps=10000
)


### Trainer

In [124]:
trainer = pl.Trainer(
    max_epochs=100,
    gpus=1 if torch.cuda.is_available() else 0,
    callbacks=[pl.callbacks.EarlyStopping('mlm/valid_mlm_loss')],
    enable_progress_bar=True,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [125]:
%%time
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, train_dl)
print(trainer.logged_metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params
-----------------------------------------------------
0 | trx_encoder     | TrxEncoder       | 5.8 K 
1 | _seq_encoder    | RnnEncoder       | 6.4 K 
2 | fn_norm_predict | PBShell          | 0     
3 | loss_fn         | QuerySoftmaxLoss | 0     
4 | train_mlm_loss  | MeanMetric       | 0     
5 | valid_mlm_loss  | MeanMetric       | 0     
-----------------------------------------------------
12.2 K    Trainable params
0         Non-trainable params
12.2 K    Total params
0.049     Total estimated model params size (MB)


logger.version = 124


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\trainer.py:1933: PossibleUserWarning: The number of training batches (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

{'mlm/loss': tensor(0.5165), 'mlm/valid_mlm_loss': tensor(0.5046), 'mlm/train_mlm_loss': tensor(0.5087)}
CPU times: total: 1min 2s
Wall time: 1min 4s


In [126]:
model_downstream = NextItemPredictionModule(
    trx_encoder=model.trx_encoder, #TrxEncoder(**trx_encoder_params),
    seq_encoder=model._seq_encoder, # seq_encoder,
    target_col='small_group',
    max_lr=0.1,
    total_steps=1000
)

In [127]:
trainer = pl.Trainer(
    max_epochs=100,
    gpus=1 if torch.cuda.is_available() else 0,
    callbacks=[pl.callbacks.EarlyStopping('gpt/valid_f1_weighted', mode='max', patience=5)],
    enable_progress_bar=True,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [128]:
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model_downstream, train_dl)
print(trainer.logged_metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | trx_encoder    | TrxEncoder       | 5.8 K 
1 | _seq_encoder   | RnnEncoder       | 6.4 K 
2 | head           | Head             | 24.9 K
3 | loss           | CrossEntropyLoss | 0     
4 | train_gpt_loss | MeanMetric       | 0     
5 | valid_gpt_loss | MeanMetric       | 0     
----------------------------------------------------
37.0 K    Trainable params
0         Non-trainable params
37.0 K    Total params
0.148     Total estimated model params size (MB)


logger.version = 125


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\trainer.py:1933: PossibleUserWarning: The number of training batches (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

{'gpt/loss': tensor(2.8960), 'gpt/valid_gpt_loss': tensor(2.9300), 'gpt/valid_f1_weighted': tensor(0.1933, dtype=torch.float64), 'gpt/train_gpt_loss': tensor(2.9312)}


In [129]:
trainer.test(model_downstream, train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
  gpt/test_f1_weighted      0.1948535026414224
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'gpt/test_f1_weighted': 0.1948535026414224}]

In [107]:
from ptls.nn import TrxEncoder, RnnEncoder
# from ptls.frames.gpt import GptPretrainModule


trx_encoder_params = dict(
    embeddings_noise=0.0,
    # numeric_values={'amount_rur': 'identity'},
    embeddings={
        # 'trans_date': {'in': 800, 'out': 16},
        'small_group': {'in': 350, 'out': 16},
        'amount_rur':{'in': 50, 'out': 16}
    },
)

seq_encoder = RnnEncoder(
        input_size=32,
        hidden_size=32,
        type='gru',
)

# model = GptPretrainModule(
#     trx_encoder=TrxEncoder(**trx_encoder_params),
#     seq_encoder=seq_encoder,
#  #   loss_type='contrast'
# )

model = MLMPretrainModule(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    seq_encoder=seq_encoder,
    total_steps=30000,
    max_lr=1,
    neg_count=5,
    replace_proba=0.15
)

### Data loader

In [108]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.gpt import GptDataset
from ptls.frames.bert import MlmDataset
from ptls.frames import PtlsDataModule


# train_dl = PtlsDataModule(
#     train_data=GptDataset(
#         MemoryMapDataset(
#             data=train,
#             # i_filters=[
#             #     SeqLenFilter(min_seq_len=25),
#             # ],
#         ),
#         min_len=25, 
#         max_len=300
#     ),
#     valid_data=GptDataset(
#         MemoryMapDataset(
#             data=test,
#             # i_filters=[
#             #     SeqLenFilter(min_seq_len=25),
#             # ],
#         ),
#         min_len=25, 
#         max_len=300
#     ),
#     train_batch_size=128,
# )

train_dl = PtlsDataModule(
    train_data=MlmDataset(
        MemoryMapDataset(
            data=train,
            i_filters=[
                SeqLenFilter(min_seq_len=25),
            ],
        ),
        min_len=25, 
        max_len=300
    ),
    valid_data=MlmDataset(
        MemoryMapDataset(
            data=test,
            i_filters=[
                SeqLenFilter(min_seq_len=25),
            ],
        ),
        min_len=25, 
        max_len=300
    ),
    train_batch_size=64,
)

### Trainer

In [109]:
import torch
import pytorch_lightning as pl

import logging

trainer = pl.Trainer(
    max_epochs=100,
    gpus=1 if torch.cuda.is_available() else 0,
    enable_progress_bar=True,
    accelerator='cpu'
)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\trainer.py:1814: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


### Training 

In [110]:
%%time
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, train_dl)
print(trainer.logged_metrics)


  | Name            | Type             | Params
-----------------------------------------------------
0 | trx_encoder     | TrxEncoder       | 6.4 K 
1 | _seq_encoder    | RnnEncoder       | 6.4 K 
2 | fn_norm_predict | PBShell          | 0     
3 | loss_fn         | QuerySoftmaxLoss | 0     
4 | train_mlm_loss  | MeanMetric       | 0     
5 | valid_mlm_loss  | MeanMetric       | 0     
-----------------------------------------------------
12.8 K    Trainable params
0         Non-trainable params
12.8 K    Total params
0.051     Total estimated model params size (MB)


logger.version = 97


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

{'mlm/loss': tensor(1.4308), 'mlm/valid_mlm_loss': tensor(1.4440), 'mlm/train_mlm_loss': tensor(1.4410)}
CPU times: total: 6min 8s
Wall time: 3min 19s


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


### Save sequence encoder for other experiments

In [ ]:
torch.save(seq_encoder.state_dict(), "coles-emb.pt")